In [ ]:
from the_well.data import WellDataset, DeltaWellDataset

from pathlib import Path
from the_well.data import WellDataModule
import matplotlib.pyplot as plt
import logging
from autoemulate.experimental.emulators.the_well import TheWellFNO

logger = logging.getLogger()

In [ ]:
from autoemulate.experimental.data.spatiotemporal_dataset import AutoEmulateDataModule, BOUTDataset

# Note this assumes that the BOUT++ data is stored in ../data/bout/ with the structure:
# ../data/bout/train/data.pt
# ../data/bout/valid/data.pt
# ../data/bout/test/data.pt
ae_data_module = AutoEmulateDataModule(
    data_path="../data/bout/",
    dataset_cls=BOUTDataset,
    verbose=False
)
output_path = Path("../data/the_well/runs/bout")

In [ ]:
# Plot example
batch = next(iter(ae_data_module.val_dataloader()))
plt.imshow(batch["input_fields"][0, 0, :, :, 0])
plt.show()

In [ ]:
# Initialize the emulator
em = TheWellFNO(datamodule=ae_data_module, output_path=output_path, device="cpu")

In [ ]:
# Fit the model
em.fit()

In [ ]:
# Validation loop
valid_results = em.trainer.validation_loop(
    ae_data_module.rollout_val_dataloader(),
    valid_or_test="rollout_valid",
    full=True
)

In [ ]:
test_results = em.trainer.validation_loop(
    ae_data_module.rollout_test_dataloader(),
    valid_or_test="rollout_test",
    full=True
)

In [ ]:
from pprint import pprint
pprint(valid_results)

In [ ]:
pprint(test_results)

In [ ]:
# Run prediction from a dataloader
em.predict(ae_data_module.rollout_test_dataloader()).shape

In [ ]:
# Run prediction from a non-rollout dataloader
em.predict(ae_data_module.test_dataloader()).shape


In [ ]:
# Initialize a UNet emulator
from autoemulate.experimental.emulators.the_well import TheWellUNetClassic


em = TheWellUNetClassic(datamodule=ae_data_module, output_path=output_path, device="cpu")

In [ ]:
em.fit()

In [ ]:
em.predict(ae_data_module.rollout_test_dataloader()).shape